# Capstone Project - Paris Neighborhoods

In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


import types
from botocore.client import Config
import ibm_boto3

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

from bs4 import BeautifulSoup
import requests

print('Libraries imported.')

Libraries imported.


In [19]:
address = 'Paris'

geolocator = Nominatim(user_agent="pa_explorer")
location = geolocator.geocode('address')
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Paris are {}, {}.'.format(latitude, longitude))
body = client_017a6e8a470f4e828f4f58738b78ab0a.get_object(Bucket='machinelearningwithpython-donotdelete-pr-z2ho74dny8ed6r',Key='quartier_paris.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

# If you are reading an Excel file into a pandas DataFrame, replace `read_csv` by `read_excel` in the next statement.
df_data = pd.read_csv(body, sep=';')
df_data.head()


The geograpical coordinate of Paris are -37.9025717, 144.75777994409498.


,N_SQ_QU,C_QU,C_QUINSEE,L_QU,C_AR,N_SQ_AR,PERIMETRE,SURFACE,Geometry X Y,Geometry
0,750000021,21,7510601,Monnaie,6,750000006,2391.122818,2.933606e+05,"48.8543844036,2.34003537113","{""type"": ""Polygon"", ""coordinates"": [[[2.343168..."
1,750000022,22,7510602,Odéon,6,750000006,3516.314464,7.161484e+05,"48.8478006293,2.33633882759","{""type"": ""Polygon"", ""coordinates"": [[[2.336988..."
2,750000029,29,7510801,Champs-Elysées,8,750000008,5480.652870,1.140254e+06,"48.8670744922,2.30865168468","{""type"": ""Polygon"", ""coordinates"": [[[2.306463..."
3,750000051,51,7511303,Maison-Blanche,13,750000013,6814.373288,2.231506e+06,"48.8231278057,2.35243314954","{""type"": ""Polygon"", ""coordinates"": [[[2.349115..."
4,750000052,52,7511304,Croulebarbe,13,750000013,3289.230480,6.920677e+05,"48.8337336761,2.34767304607","{""type"": ""Polygon"", ""coordinates"": [[[2.351662..."


In [77]:
paris_data = df_data.loc[:,['C_AR','Geometry X Y','C_QUINSEE','L_QU']]
paris_data['C_QUINSEE'] = paris_data['C_QUINSEE'].astype(str)
paris_data['code'] = paris_data['C_QUINSEE'].str[0:5].astype(str)
paris_data['Latitude'] = paris_data['Geometry X Y'].str.extract('(.*)\,')
paris_data['Longitude'] = paris_data['Geometry X Y'].str.extract(',(.*)')
paris_data = paris_data.drop(['Geometry X Y','C_QUINSEE'],axis=1)
paris_data.columns= ['Borough', 'Neighbourhood', 'Code', 'Latitude','Longitude']
paris_data = paris_data.reset_index()

paris_data['Borough'] = paris_data['Borough'].astype(str)
paris_data['Code'] = paris_data['Code'].astype(str)
paris_data['Latitude'] = paris_data['Latitude'].astype(float)
paris_data['Longitude'] = paris_data['Longitude'].astype(float)

paris_data.head()

,index,Borough,Neighbourhood,Code,Latitude,Longitude
0,0,6,Monnaie,75106,48.854384,2.340035
1,1,6,Odéon,75106,48.847801,2.336339
2,2,8,Champs-Elysées,75108,48.867074,2.308652
3,3,13,Maison-Blanche,75113,48.823128,2.352433
4,4,13,Croulebarbe,75113,48.833734,2.347673


In [78]:
# create map of Toronto using latitude and longitude values
longitude = 2.3488

latitude = 48.85341
 
map_paris = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, codepost, neighborhood, borough in zip(paris_data['Latitude'], paris_data['Longitude'], paris_data['Code'], paris_data['Neighbourhood'], paris_data['Borough']):
    label = '{},{}, {} ème'.format(codepost, neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_paris) 

map_paris

In [61]:
CLIENT_ID = 'AAXS11JUTRNPPC4LYAOQYLTGYUAH13CIHKHXHJEQJY4ERUDM' # your Foursquare ID
CLIENT_SECRET = '0QDSFHVBM3Z1DPVSSWXMCESYJNHNPPJMRT4OJUGZV3IFNDC0' # your Foursquare Secret
VERSION = '20200210' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: AAXS11JUTRNPPC4LYAOQYLTGYUAH13CIHKHXHJEQJY4ERUDM
CLIENT_SECRET:0QDSFHVBM3Z1DPVSSWXMCESYJNHNPPJMRT4OJUGZV3IFNDC0


In [79]:
neighborhood_latitude = paris_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = paris_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = paris_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Monnaie are 48.8543844036, 2.34003537113.


In [80]:
# type your answer here
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

In [81]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e442a1bd03993001b63cf52'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Paris',
  'headerFullLocation': 'Paris',
  'headerLocationGranularity': 'city',
  'totalResults': 151,
  'suggestedBounds': {'ne': {'lat': 48.8588844081, 'lng': 2.346861774269252},
   'sw': {'lat': 48.8498843991, 'lng': 2.333208967990748}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b22eca6f964a5206c5024e3',
       'name': 'Prescription Cocktail Club',
       'location': {'address': '23 rue Mazarine',
        'lat': 48.854921,
        'lng': 2.337768,
        'labeledLatLngs': [{'label': 'display',
          'lat': 48.854921,
          'lng': 2.337768}],
        'd

In [82]:
results['response']['groups'][0].keys()

dict_keys(['type', 'name', 'items'])

In [83]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [84]:
from pandas.io.json import json_normalize

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Prescription Cocktail Club,Cocktail Bar,48.854921,2.337768
1,Chez Nous,Wine Bar,48.855823,2.340271
2,Maison Sauvage,French Restaurant,48.853744,2.337993
3,Chez Le Libanais,Lebanese Restaurant,48.853285,2.341673
4,Ze Kitchen Galerie,Restaurant,48.855101,2.341297


In [85]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


In [87]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [88]:
paris_venues = getNearbyVenues(names=paris_data['Neighbourhood'],
                                   latitudes=paris_data['Latitude'],
                                   longitudes=paris_data['Longitude']
                                  )

paris_venues.head()

Monnaie
Odéon
Champs-Elysées
Maison-Blanche
Croulebarbe
Sorbonne
Saint-Georges
Chaussée-d'Antin
Palais-Royal
Saint-Merri
Notre-Dame
Gros-Caillou
Arsenal
Jardin-des-Plantes
Porte-Saint-Martin
Roquette
Picpus
Plaisance
Saint-Victor
Madeleine
Saint-Fargeau
Porte-Dauphine
Grandes-Carrières
Batignolles
Europe
Sainte-Marguerite
Parc-de-Montsouris
Saint-Lambert
Quinze-Vingts
Ecole-Militaire
Grenelle
Auteuil
Folie-Méricourt
Salpêtrière
Place-Vendôme
Combat
Charonne
Javel
Val-de-Grâce
Necker
Père-Lachaise
La Chapelle
Arts-et-Métiers
Archives
Faubourg-du-Roule
Rochechouart
Bercy
Halles
Notre-Dame-des-Champs
Petit-Montrouge
Pont-de-Flandre
Muette
Chaillot
Epinettes
Mail
Bonne-Nouvelle
Gare
Clignancourt
Goutte-d'Or
Vivienne
Enfants-Rouges
Saint-Germain-des-Prés
Saint-Vincent-de-Paul
Saint-Ambroise
Bel-Air
Montparnasse
Plaine de Monceaux
Saint-Gervais
Saint-Thomas-d'Aquin
Porte-Saint-Denis
Saint-Germain-l'Auxerrois
Villette
Sainte-Avoie
Hôpital-Saint-Louis
Belleville
Ternes
Invalides
Faubourg-Montm

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Monnaie,48.854384,2.340035,Prescription Cocktail Club,48.854921,2.337768,Cocktail Bar
1,Monnaie,48.854384,2.340035,Chez Nous,48.855823,2.340271,Wine Bar
2,Monnaie,48.854384,2.340035,Maison Sauvage,48.853744,2.337993,French Restaurant
3,Monnaie,48.854384,2.340035,Chez Le Libanais,48.853285,2.341673,Lebanese Restaurant
4,Monnaie,48.854384,2.340035,Ze Kitchen Galerie,48.855101,2.341297,Restaurant


In [93]:
paris_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Amérique,14,14,14,14,14,14
Archives,100,100,100,100,100,100
Arsenal,71,71,71,71,71,71
Arts-et-Métiers,100,100,100,100,100,100
Auteuil,16,16,16,16,16,16
Batignolles,100,100,100,100,100,100
Bel-Air,6,6,6,6,6,6
Belleville,44,44,44,44,44,44
Bercy,76,76,76,76,76,76


In [100]:
# one hot encoding
paris_onehot = pd.get_dummies(paris_venues[['Venue Category']], prefix="", prefix_sep="")
# add neighborhood column back to dataframe
paris_onehot['Neighbourhood'] = paris_venues['Neighbourhood'] 


# move neighborhood column to the first column
fixed_columns = [paris_onehot.columns[-1]] + list(paris_onehot.columns[:-1])
paris_onehot = paris_onehot[fixed_columns]

paris_onehot.head()

,Neighbourhood,Accessories Store,Afghan Restaurant,African Restaurant,Alsatian Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Auvergne Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Basque Restaurant,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Boxing Gym,Brasserie,Brazilian Restaurant,Breakfast Spot,Breton Restaurant,Brewery,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burgundian Restaurant,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Canal,Canal Lock,Candy Store,Cantonese Restaurant,Cemetery,Ch'ti Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Clothing Store,Cocktail Bar,Coffee Shop,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Corsican Restaurant,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cultural Center,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Dive Bar,Dive Spot,Doner Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hawaiian Restaurant,Health Food Store,Herbs & Spices Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Insurance Office,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Jiangxi Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Leather Goods Store,Lebanese Restaurant,Lingerie Store,Liquor Store,Locksmith,Lounge,Lyonese Bouchon,Mac & Cheese Joint,Market,Martial Arts Dojo,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Multiplex,Museum,Music Store,Music Venue,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,Office,Okonomiyaki Restaurant,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Peruvian Restaurant,Pet Café,Pet Store,Pharmacy,Pizza Place,Planetarium,Platform,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Provençal Restaurant,Pub,Racecourse,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Resort,Restaurant,Rock Club,Romanian Restaurant,Roof Deck,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Savoyard Restaurant,Scandinavian Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shanxi Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Smoke Shop,Snack Place,Soba Restaurant,Soccer Field,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Southwestern French Restaurant,Souvenir Shop,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,S